## CSV dosyasındaki verinin yüklenmesini gerçekleştirme

In [ ]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "IMDb_Top_1000_Movies_Dataset.csv"
loader = CSVLoader(filepath, encoding="utf-8")

data = loader.load()

print(f"Toplam {len(data)} film yüklendi.")

## Veriyi parçalarına ayırma(Chunking işlemi)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

print(f"Number of documents after splitting: {len(docs)}")
docs[7]

## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [ ]:
load_dotenv()  # Load environment variables from .env file
import os
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    print("HATA: API anahtarı .env dosyasından okunamadı!")
else:
    print(f"Anahtar başarıyla yüklendi. İlk 4 karakter: {api_key[:4]}")

# Huggingface alternatifi

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector = embeddings.embed_query("hello, world!")
vector[:5]


In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

## ChromaDB üzerine kayıt işlemi

In [ ]:
from langchain_chroma import Chroma

In [ ]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5 })

In [ ]:
retrieved_docs = retriever.invoke("high budget expensive movies")     

len(retrieved_docs)

print(retrieved_docs[5].page_content) 


## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [ ]:
system prompt = (
    "You are assistant for question-answering tasks."
    "Use the following context to answer the question at the end."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "\n\n"
    "{context}"
)



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [ ]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)


## RAG Zinciri Oluşturma (RAG + LLM)


In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

## Kullanıcı sorgusunu çalıştırma

In [ ]:
response = rag_chain.invoke({"input": "Explain the transformer architecture?"})

print(response["answer"])